In [4]:
from pyspark.context import SparkContext
from pyspark import SparkConf,sql
from pyspark.sql.session import SparkSession
import findspark
import pymysql
import pymysql.cursors
import pandas as pd
from pyspark.sql.functions import *

In [75]:
product_path='D:/PROJECTS/Recomendation_System_FP/Code/Database/product.csv'
cosin_result_path1='D:/PROJECTS/Recomendation_System_FP/Code/cosin_similarity_registry/cosin_similarity.csv'


In [76]:
spark = SparkSession.builder \
    .appName("PySpark MySQL Example") \
    .config("spark.driver.extraClassPath", "C:/spark/spark-3.5.0-bin-hadoop3/jars/mysql-connector-java-5.1.46.jar") \
    .getOrCreate()



In [95]:
Product_df=spark.read.format('csv').option('header',True) \
.load(product_path)

cosin_result_path_df1=spark.read.format('csv').option('header',True) \
.load(cosin_result_path1)



In [78]:
# Product_df=Product_df.distinct()

In [79]:
# len(Product_df.select('product_id').collect())

4324

In [107]:
cosin_date=Product_df.join(cosin_result_path_df1,cosin_result_path_df1.original_product==Product_df.product_id,'inner')
cosin_date=cosin_date.withColumnRenamed('published_at','reference_date').select(['original_product','matched_product','reference_date','cosin_score'])
cosin_date=cosin_date.join(Product_df,Product_df.product_id==cosin_date.matched_product,'inner').select(['original_product','matched_product','reference_date','cosin_score','published_at'])

In [97]:
cosin_date.show()

+----------------+---------------+-------------------+-----------+-------------------+
|original_product|matched_product|     reference_date|cosin_score|       published_at|
+----------------+---------------+-------------------+-----------+-------------------+
|            4821|           4894|2021-10-04 18:56:21|      0.979|2021-10-21 12:56:20|
|            4821|           7650|2021-10-04 18:56:21|      0.914|2023-06-27 08:10:35|
|            4821|           3266|2021-10-04 18:56:21|      0.974|2019-11-26 00:00:00|
|            4821|           5234|2021-10-04 18:56:21|       0.97|2022-01-19 10:38:35|
|            4821|           5408|2021-10-04 18:56:21|      0.902|2022-03-18 09:54:57|
|            4821|           8279|2021-10-04 18:56:21|      0.952|2023-10-10 07:27:36|
|            4821|           2644|2021-10-04 18:56:21|      0.846|2019-06-27 21:05:20|
|            4821|           4417|2021-10-04 18:56:21|      0.869|2021-06-13 12:18:17|
|            4821|           4690|2021-10-0

In [10]:
from pyspark.sql.functions import datediff,to_date,col

In [108]:
cosin_date=cosin_date.withColumn('datediff',datediff(to_date('reference_date'),to_date('published_at')))

In [109]:
cosin_date=cosin_date.withColumn('cosin_score_2',col('cosin_score')*col('datediff'))

In [110]:
cosin_date=cosin_date.select(['original_product','matched_product','cosin_score_2'])

In [111]:
cosin_date=cosin_date.withColumnRenamed('cosin_score_2','cosin_score')

In [112]:
cosin_date=cosin_date.select('*',round('cosin_score',3).alias('cosin_score_1'))

In [113]:
cosin_date=cosin_date.select(['original_product','matched_product','cosin_score_1'])

In [114]:
cosin_date=cosin_date.withColumnRenamed('cosin_score_1','cosin_score')

In [118]:
cosin_date.show()

+----------------+---------------+-----------+
|original_product|matched_product|cosin_score|
+----------------+---------------+-----------+
|            4821|           4894|    -16.643|
|            4821|           7650|   -576.734|
|            4821|           3266|    660.372|
|            4821|           5234|    -103.79|
|            4821|           5408|    -148.83|
|            4821|           8279|   -700.672|
|            4821|           2644|     702.18|
|            4821|           4417|     98.197|
|            4821|           4690|      29.44|
|            4821|           3848|     262.44|
|            4821|           5543|    -183.38|
|            4821|           4980|    -88.776|
|            4821|           4934|     -40.35|
|            4821|           7539|   -613.104|
|            4821|           7689|   -614.705|
|            4821|           2605|    772.719|
|            4821|           5412|   -163.305|
|            4821|           3570|    559.411|
|            

In [106]:
# cosin_date=cosin_date.toPandas()

In [116]:
cosin_date=cosin_date[['original_product','matched_product','cosin_score']]

In [121]:
len(np.unique(cosin_date.select('original_product').collect()))

4163

In [59]:
# cosin_date.to_csv('cosin_similarity.csv',index=False)

In [119]:
# len(np.unique(cosin_date['original_product'].values))

1

In [78]:
# cosin_date.to_csv('cosin_similarity_data_diff.csv')

In [122]:
cosin_result_path_df4=cosin_date

In [127]:
product_title='RIBCAGE SHORT'
selected_product_id=Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]
recomendation_df_product=cosin_result_path_df4[cosin_result_path_df4['original_product']==selected_product_id]
recomendation_df_product=recomendation_df_product.sort(col('cosin_score').asc())
recomendation_df_product.show()


+----------------+---------------+-----------+
|original_product|matched_product|cosin_score|
+----------------+---------------+-----------+
|            4535|           8666|     -88.74|
|            4535|           8655|    -70.602|
|            4535|           8624|    -54.351|
|            4535|           8163|    -49.952|
|            4535|           8545|    -40.896|
|            4535|           8587|    -40.044|
|            4535|           8588|    -40.044|
|            4535|           8541|     -38.28|
|            4535|           8162|    -36.636|
|            4535|           8369|    -35.862|
|            4535|           8434|    -34.788|
|            4535|           6698|    -20.982|
|            4535|           8335|    -18.656|
|            4535|           8093|      2.556|
|            4535|           8207|      6.672|
|            4535|           7928|     14.824|
|            4535|           8094|      18.27|
|            4535|           7895|     18.963|
|            

In [126]:
link=[]
print(selected_product_id)
for row in recomendation_df_product.collect():
    if float(row['cosin_score']) < 1 :
        id=row['matched_product']
        # datediff=row['datediff']
        # score=row['cosin_score']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
            link.append(id)
        except:
            print('valid string')    
print(','.join(link))

5174
8628,8510,8441,8612,8452,8381,8399,8065,8382,8458,8327,8488,8281,8294,7859,7840,8384,7973,8373,8249,8340,7951,8146,8445,8459,7858,8210,7857,7374,7330,7127,6674,6706,6671,6702,6058,6404,6365,6235,5594,5409,5147,5153,5108,5176


In [59]:
selected_product_id=Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]
recomendation_df_product=cosin_result_path_df2[cosin_result_path_df2['Product_A']==selected_product_id]
link=[]
for row in recomendation_df_product.collect():
    if float(row['cosin_score'])>=.80:
        id=row['Product_B']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
             link.append(id)
        except:
            print('valid string')    
print(','.join(link))

3939,4407,7137,8167,2668,6832,8202,4815,8245,7534,8570,2006,5441,4286,7811,5091,2459,7998,5194,5943,8477,2591,5602,5032,6224,6845,8552,7641,3962,8081,1976,4627,2687,5860,3963,2844,8331,3323,7141,3623,8476,4811,4900,7733,8753,4946,7960,4285,8097,7804,7726,5172,7844,7803,1951,4331,4311,7725,3769,4810,1231,2328,7308,5940,8143,5382,8250,7661,6798,5410,5938,6283,8080,3961,6223,6678,2177,7826,4962,4108,4406,2254,8051,8637,6890,3192,7148,3217,5423,7602,8752,5381,8550,8750,3564,8664,4927,7997,5438,4143,8254,7535,4625,5736,180,4102,1531,316,6432,1450,5201,5719,5741,6975,6231,7619,5939,8316,8076,2549,8075,3759,5424,5422,5368,3878,5405,5288,5611,859,6225,6073,3440,1952,3423,5404,5369,5182,4963,7140,7986,2332,4978,7876,4210,7800,6540,4287,3205,3575,7802,7796,1949,5279,1614,7938,5089,8754,3385,557,8475,2001,915,6421,5858,6610,3749,3725,5281,8487,5126,5171,4145,2399,7172,7660,2707,5185,7943,8751,2114,2029,5131,6265,278,6115,3396,8054,8867,8400,4409,3422,8068,7124,8523,6397,6880,8660,1251


In [8]:
selected_product_id=Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]
recomendation_df_product=cosin_result_path_df3[cosin_result_path_df3['Product_A']==selected_product_id]
link=[]
for row in recomendation_df_product.collect():
    if float(row['cosin_score'])>=.80:
        id=row['Product_B']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
             link.append(id)
        except:
            print('valid string')    
print(','.join(link))

5444,2324,2940,7779,1668,7630,1200,2747,604,104,3599,978,3713,5342,7675,5973,3921,1557,7555,7874,8112,4763,1790,645,5506,1237,1301,1105,1078,8033,3473,7253,3538,1348,1201,8405,1028,7599,1714,1068,5446,7748,1206,1523,7276,5749,8403,1558,7870,8155,4188,7557,7608,281,461,741,1080,350,114,2897,540,759,8192,4958,1597,8032,7965,1768,3598,8102,490,24,7558,757,3356,8141,7744,5391,1778,289,8156,5448,7443,5447,8132,7254,1848,2924,8231,776,3596,3675,5670,8404,7783,3514,49,1661,4425,7610,7265,1242,2320,4181,4254,3710,7871,1233,1029,4251,1553,7206,4486,1350,1273,66,8191,8232,760,2730,8230,8401,5389,8506,7216,1699,6718,1781,2704,3594,5390,7264,8111,2746,3601,2732,5953,4316,145,2076,8188,6137,881,1779,2317,119,8128,8190,1329,8189


In [7]:
import pandas as pd
cs1=pd.read_csv(cosin_result_path3)#configur.get('cosin_similarity_base_path','cs1'))#load_csv(spark_session,file_path=configur.get('cosin_similarity_base_path','cs1'))
cs2=pd.read_csv(cosin_result_path2)#configur.get('cosin_similarity_base_path','cs2'))#load_csv(spark=spark_session,file_path=configur.get('cosin_similarity_base_path','cs2'))
cs3=pd.read_csv(cosin_result_path1)

In [50]:
product_title='X CAMELIA FARHOODI THE LEILA CHOKER'
selected_product_id=[Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]]
(selected_product_id[0])

'6878'

In [12]:
# from Code.utils.libraries import * 
# from Code.data.dataprocessing.load.load import *
# from Code.data.dataprocessing.transform.transform import *
# from sessions import *
# from pyspark.sql.functions import date_format
# from Code.models.model_training.model import *


# BASE_PATH=os.getenv('BASEPATH')
# CONN_URL = os.getenv('CONN_URL')
# PASSWORD = os.getenv('PASSWORD')
# USERNAME=os.getenv('USER_NAME')
# configur = ConfigParser() 
# configur.read(BASE_PATH+'config.ini')
#spark_session=session(configur.get('driver_path_pyspark','pyspark'))
#configur.get('cosin_similarity_base_path','cs3'))#Sload_csv(spark=spark_session,file_path=configur.get('cosin_similarity_base_path','cs3'))

cosin_df=[]
product_title='SIENA TOP'
product_ids=[Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]]#[(Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0])]#cs1.Product_A.unique()#[id[0] for id in cs1.select('Product_A').distinct().collect()]  in pyspark [8692,2178,8687,1087,3069]
print("total number of product",len(product_ids))
# st = time.time()

for r in product_ids:
    r=int(r)
    print(r)
    cs1_product_b=[]
    cs2_product_b=[]
    cs3_product_b=[]

    cs1_product_b=cs1[(cs1['Product_A']==r) & (cs1['cosin_score']>=.80)]['Product_B'].unique().tolist()  #[ product[0] for product in cs1.filter((cs1['Product_A']==r) & (cs1['cosin_score']>=.80))['Product_B'].unique()]
    cs2_product_b=cs2[(cs2['Product_A']==r) & (cs2['cosin_score']>=.85)]['Product_B'].unique().tolist()#values
    cs3_product_b=cs3[(cs3['Product_A']==r) & (cs3['cosin_score']>=.90)]['Product_B'].unique().tolist()#values
    # cs2temp=cs2.filter((cs2['Product_A']==r) & (cs2['cosin_score']>=.85))
    # cs3temp=cs3.filter((cs3['Product_A']==r) & (cs3['cosin_score']>=.90))
    # print(cs1_product_b)
    # print(cs2_product_b)
    # print(cs3_product_b)

    # for row in  cs1temp.collect():
    #     if float(row['cosin_score'])>=.80:
    #         cs1_product_b.append(row['Product_B'])
        
    # for row2 in cs2temp.collect():
    #     if float(row2['cosin_score'])>=.85:
    #         cs1_product_b.append(row2['Product_B'])
    #         # cs2_product_b.append(row2['Product_B'])
    # for row3 in cs3temp.collect():
    #     if float(row3['cosin_score'])>=.90 :
    #         cs1_product_b.append(row3['Product_B'])
                # cs3_product_b.append(row3['Product_B'])

#print(cs1_product_b,cs2_product_b,cs3_product_b)
    for id in cs2_product_b:
        cs1_product_b.append(id)
    for id in cs3_product_b:
        cs1_product_b.append(id)
    # final_list=cs1_product_b+cs2_product_b+cs3_product_b
    my_dict = {i:cs1_product_b.count(i) for i in cs1_product_b}
    # print(my_dict)
    top_recomendation=[]
    for key,value in my_dict.items():
        if value >=3:
            top_recomendation.append(key)
    print('top re',top_recomendation)
    cosin_df.append(cs1[(cs1['Product_A']==r) & (cs1['Product_B'].isin(top_recomendation))])
    # cosin_df.append(cs1.filter((cs1['Product_A']==r) & (cs1['Product_B'].isin(top_recomendation))))  # in pandas

df=cosin_df[0]
print(len(cosin_df[1:]))
for i in cosin_df[1:]:
    #  try:
         #df=df.union(i) #in pyspark
         df=df._append(i,ignore_index = True) #in pandas
    #  except:
    #      print('out of index')
#df.toPandas().to_csv('cosin_simmilarity1.csv')
# et = time.time()
# print(f'Execution time:{et-st} seconds')
# df.to_csv('cosin_simmilarity2.csv')






total number of product 1
7955
top re []
0
